In [4]:
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import pandas as pd
import numpy as np 
import matplotlib as plt
import seaborn as sns

In [5]:
# read cv file
data = pd.read_csv('../stilligo_GitHub/Russian tourist_attractions.csv')
df = data

In [53]:
######## Transformation of latitude and longitude columns ##########
df = data
# slipe coodinate data (geolocalisation column) into "latitude" and "longitude" columns
df[['longitude', 'latitude']] = df['geolocation'].str.strip('()').str.split(', ', expand=True)

# Remove non-numeric characters and convert to numeric type
df['longitude'] = df['longitude'].str.replace(r'[^0-9. ]', '', regex=True).astype(str)
df['latitude']  = df['latitude'].str.replace(r'[^0-9. ]', '', regex=True).astype(str)

# change latitude and longitude type from str to float step
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

# empty rows droped
df = df.dropna()
df = df.drop_duplicates()

#garder juste la colonne desiré
columns_to_drop = ['geolocation', 'name']  # List of columns to drop
df = df.drop(columns=columns_to_drop)

df.head()



,type,region,locality,longitude,latitude
0,architecture,Amur region,Blagoveshchensk,127.548877,49.811568
1,architecture,Sverdlovsk region,Ekaterinburg,60.621271,56.841798
2,architecture,Kursk region,Safonovka,35.139965,51.491293
3,architecture,Tomsk region,Tomsk,85.050946,56.469513
4,architecture,Novosibirsk region,Novosibirsk,82.958761,55.041787


In [95]:
# count of unique values in each column
print(df['type'].nunique())

27


In [94]:
# count of each unique value in the "Gender" column
print(df['type'].value_counts())

architecture                        1213
Monuments, sculptures, memorials    1134
military glory                       366
monasteries                          260
Granges                              117
archaeological sites                  98
defenses                              44
palaces                               35
Obelisk / Stele                       26
Abandoned shrine                      24
Abandoned places                      22
Ruin                                  16
Gardens, squares, parks               11
galleries                              9
places of pilgrimage                   6
Nature                                 6
Cemeteries and graveyards              3
Personal, memorial museum              3
monuments Railways                     2
Other museums                          2
industrial facilities                  1
Entertainment                          1
memorial complexes                     1
tourist camps                          1
Farmhouse       

In [88]:
####### pipeline  #######
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# Impute then scale numerical values: 
num_transformer = Pipeline([
    ('OrdinalEncoder', OrdinalEncoder()), ('MinMaxScaler', MinMaxScaler())
])


# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False)

# Parallelize "num_transformer" and "cat_transfomer"
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['region','locality']),
    ('cat_transformer', cat_transformer, ['type'])
],  remainder='passthrough')# to get the others columns without transformation

In [89]:
 # Visualizing Pipelines in HTML
from sklearn import set_config; set_config(display='diagram')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer',
                                 Pipeline(steps=[('OrdinalEncoder',
                                                  OrdinalEncoder()),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler())]),
                                 ['region', 'locality']),
                                ('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['type'])])

In [82]:
# Defining the features and the target

#example model
# X = data.drop(columns='charges')  
# y = data['charges']
X = df.drop(columns=['latitude','longitude'], axis = 1)
y = df[['latitude','longitude']]




In [83]:
# Train-Test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2723, 3), (681, 3), (2723, 2), (681, 2))

In [90]:
#fit the pipeline to the training data
X_train_transformed = preprocessor.fit_transform(X_train)

print("Original training set")
display(X_train.head(3))

print("Preprocessed training set")
display(pd.DataFrame(X_train_transformed).head(5))

Original training set


/home/gilles/.pyenv/versions/3.8.12/envs/stilligo/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,type,region,locality
3715,Nature,Chechen Republic,Xaracoj
1171,architecture,Novosibirsk region,Novosibirsk
43,architecture,Krasnodar region,Hot key


Preprocessed training set


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.084337,0.861111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.457831,0.482323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.289157,0.215909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.433735,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.710843,0.695707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [96]:
# count of unique values in each column
print(X_train['type'].nunique())

24


In [101]:
# count of each unique value in the "Gender" column
print(X_train['type'].value_counts())
val_X_train = X_train['type'].value_counts()

architecture                        985
Monuments, sculptures, memorials    884
military glory                      305
monasteries                         202
Granges                              98
archaeological sites                 77
defenses                             35
palaces                              30
Abandoned places                     20
Obelisk / Stele                      18
Abandoned shrine                     17
Ruin                                 13
Gardens, squares, parks               9
galleries                             6
places of pilgrimage                  6
Nature                                5
Personal, memorial museum             3
Cemeteries and graveyards             2
Other museums                         2
monuments Railways                    2
Farmhouse                             1
Entertainment                         1
tourist camps                         1
industrial facilities                 1
Name: type, dtype: int64


In [100]:
# count of each unique value in the "Gender" column
print(df['type'].value_counts())
val_df = df['type'].value_counts()

architecture                        1213
Monuments, sculptures, memorials    1134
military glory                       366
monasteries                          260
Granges                              117
archaeological sites                  98
defenses                              44
palaces                               35
Obelisk / Stele                       26
Abandoned shrine                      24
Abandoned places                      22
Ruin                                  16
Gardens, squares, parks               11
galleries                              9
places of pilgrimage                   6
Nature                                 6
Cemeteries and graveyards              3
Personal, memorial museum              3
monuments Railways                     2
Other museums                          2
industrial facilities                  1
Entertainment                          1
memorial complexes                     1
tourist camps                          1
Farmhouse       

In [106]:
#joint of df and X_train's type column values
print(pd.concat([val_df , val_X_train], ignore_index = True, axis =1))

                                     0      1
architecture                      1213  985.0
Monuments, sculptures, memorials  1134  884.0
military glory                     366  305.0
monasteries                        260  202.0
Granges                            117   98.0
archaeological sites                98   77.0
defenses                            44   35.0
palaces                             35   30.0
Obelisk / Stele                     26   18.0
Abandoned shrine                    24   17.0
Abandoned places                    22   20.0
Ruin                                16   13.0
Gardens, squares, parks             11    9.0
galleries                            9    6.0
places of pilgrimage                 6    6.0
Nature                               6    5.0
Cemeteries and graveyards            3    2.0
Personal, memorial museum            3    3.0
monuments Railways                   2    2.0
Other museums                        2    2.0
industrial facilities             

In [97]:
X_train

,type,region,locality
3715,Nature,Chechen Republic,Xaracoj
1171,architecture,Novosibirsk region,Novosibirsk
43,architecture,Krasnodar region,Hot key
787,architecture,Nizhny Novgorod Region,Nizhny Novgorod
4058,palaces,St. Petersburg,St. Petersburg
...,...,...,...
2289,"Monuments, sculptures, memorials",Perm,Permian
565,architecture,Sverdlovsk region,Ekaterinburg
3575,"Monuments, sculptures, memorials",Bryansk region,Bryansk
1899,military glory,Kursk region,Kursk


In [86]:
# Get features' names
preprocessor.get_feature_names_out()

array(['num_transformer__region', 'num_transformer__locality',
       'cat_transformer__type_Abandoned places',
       'cat_transformer__type_Abandoned shrine',
       'cat_transformer__type_Cemeteries and graveyards',
       'cat_transformer__type_Entertainment',
       'cat_transformer__type_Farmhouse',
       'cat_transformer__type_Gardens, squares, parks',
       'cat_transformer__type_Granges',
       'cat_transformer__type_Monuments, sculptures, memorials',
       'cat_transformer__type_Nature',
       'cat_transformer__type_Obelisk / Stele',
       'cat_transformer__type_Other museums',
       'cat_transformer__type_Personal, memorial museum',
       'cat_transformer__type_Ruin',
       'cat_transformer__type_archaeological sites',
       'cat_transformer__type_architecture',
       'cat_transformer__type_defenses',
       'cat_transformer__type_galleries',
       'cat_transformer__type_industrial facilities',
       'cat_transformer__type_military glory',
       'cat_transforme

In [87]:
pd.DataFrame(
    X_train_transformed, 
    columns=preprocessor.get_feature_names_out()
).head(6)

,num_transformer__region,num_transformer__locality,cat_transformer__type_Abandoned places,cat_transformer__type_Abandoned shrine,cat_transformer__type_Cemeteries and graveyards,cat_transformer__type_Entertainment,cat_transformer__type_Farmhouse,"cat_transformer__type_Gardens, squares, parks",cat_transformer__type_Granges,"cat_transformer__type_Monuments, sculptures, memorials",...,cat_transformer__type_architecture,cat_transformer__type_defenses,cat_transformer__type_galleries,cat_transformer__type_industrial facilities,cat_transformer__type_military glory,cat_transformer__type_monasteries,cat_transformer__type_monuments Railways,cat_transformer__type_palaces,cat_transformer__type_places of pilgrimage,cat_transformer__type_tourist camps
0,0.084337,0.861111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.457831,0.482323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.289157,0.215909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.433735,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.710843,0.695707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.710843,0.710859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [73]:
#!git add .
#!git commit -am 'first commit'
#!git push origin main

In [69]:
#score the knn model on the testing data
preprocessor.score(X_test,y_test)

AttributeError: 'ColumnTransformer' object has no attribute 'score'

In [50]:
!git add

Nothing specified, nothing added.
Maybe you wanted to say 'git add .'?


In [51]:
!git commit -m "pipeline 01"


On branch main
Your branch is behind 'origin/main' by 5 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	KNN_test01.py
	Python_rus.py
	df_geolocation.py

no changes added to commit (use "git add" and/or "git commit -a")


In [52]:
!git pull

Updating 32876bd..544821f
error: Your local changes to the following files would be overwritten by merge:
	.gitignore
Please commit your changes or stash them before you merge.
Aborting


In [ ]:
git push --help